In [1]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularizers = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 19.922598
Minibatch accuracy: 15.6%
Validation accuracy: 13.7%
Minibatch loss at step 500: 2.421427
Minibatch accuracy: 76.6%
Validation accuracy: 74.5%
Minibatch loss at step 1000: 1.981028
Minibatch accuracy: 69.5%
Validation accuracy: 77.1%
Minibatch loss at step 1500: 1.531056
Minibatch accuracy: 73.4%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 1.197384
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 2500: 1.070907
Minibatch accuracy: 75.8%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 0.950429
Minibatch accuracy: 79.7%
Validation accuracy: 79.5%
Test accuracy: 87.1%


In [7]:
# 一层神经网络 1024节点
batch_size = 128
layer1node = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]))
  layer1_biases = tf.Variable(tf.zeros([layer1node]))
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros([num_labels]))
    
   # Model.
  def model(data):
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    return tf.matmul(hidden, layer2_weights) + layer2_biases
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases)
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * regularizers
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 187.472656
Minibatch accuracy: 11.7%
Validation accuracy: 25.0%
Minibatch loss at step 500: 129.986938
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 97.164719
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 74.240959
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 56.902184
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 44.182713
Minibatch accuracy: 83.6%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 34.490967
Minibatch accuracy: 85.2%
Validation accuracy: 83.6%
Test accuracy: 90.7%


In [9]:
batch_size = 128
layer1node = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1),name='layer2')
  layer2_biases = tf.Variable(tf.zeros([layer1node]),name='biases2')
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    
   # Model.
  def model(data):
    hidden1=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    hidden2=tf.nn.relu(tf.matmul(hidden1, layer2_weights) + layer2_biases)
    return tf.matmul(hidden2, layer3_weights) + layer3_biases
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 956.808044
Minibatch accuracy: 3.1%
Validation accuracy: 18.8%
Minibatch loss at step 500: 159.916275
Minibatch accuracy: 64.1%
Validation accuracy: 66.1%
Minibatch loss at step 1000: 159.116745
Minibatch accuracy: 64.1%
Validation accuracy: 68.0%
Minibatch loss at step 1500: 158.315063
Minibatch accuracy: 68.0%
Validation accuracy: 70.3%
Minibatch loss at step 2000: 157.220490
Minibatch accuracy: 69.5%
Validation accuracy: 71.4%
Minibatch loss at step 2500: 156.622833
Minibatch accuracy: 66.4%
Validation accuracy: 71.9%
Minibatch loss at step 3000: 155.429123
Minibatch accuracy: 75.0%
Validation accuracy: 72.8%
Minibatch loss at step 3500: 154.707565
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Minibatch loss at step 4000: 154.166931
Minibatch accuracy: 81.2%
Validation accuracy: 74.1%
Minibatch loss at step 4500: 153.293106
Minibatch accuracy: 75.0%
Validation accuracy: 74.6%
Minibatch loss at step 5000: 152.480988
Minibatch accuracy: 77.

In [74]:
batch_size = 128
layer1node = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1),name='layer2')
  layer2_biases = tf.Variable(tf.zeros([layer1node]),name='biases2')
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    
   # Model.
  def model(data):
    hidden1=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    hidden2=tf.nn.relu(tf.matmul(hidden1, layer2_weights) + layer2_biases)
    return tf.matmul(hidden2, layer3_weights) + layer3_biases
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += 5e-4 * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [75]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 629.981934
Minibatch accuracy: 18.8%
Validation accuracy: 10.1%


KeyboardInterrupt: 

In [102]:
batch_size = 128
layer1node = 1024
layernum=1
rate=0.5
lossrate= 5e-4
decaystep=1000
decayrate=0.96
graph = tf.Graph()
with graph.as_default():
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(rate, global_step, decaystep,decayrate)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layermiddle_weights = [tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1)) for i in range(layernum)]
  layermiddle_biases = [tf.Variable(tf.zeros([layer1node])) for i in range(layernum)]
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    
   # Model.
  def model(data):
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    for i in range(layernum):
        hidden=tf.nn.sigmoid(tf.matmul(hidden, layermiddle_weights[i]) + layermiddle_biases[i])
    return tf.matmul(hidden, layer3_weights) + layer3_biases
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases)  + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
  for i in range(layernum):
    regularizers += (tf.nn.l2_loss(layermiddle_weights[i]) + tf.nn.l2_loss(layermiddle_biases[i]))

  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += lossrate * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [103]:
num_steps = 30001
# num_steps = 15001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 187.001373
Minibatch accuracy: 13.3%
Validation accuracy: 10.9%
Minibatch loss at step 500: 130.952255
Minibatch accuracy: 59.4%
Validation accuracy: 68.2%
Minibatch loss at step 1000: 102.784340
Minibatch accuracy: 72.7%
Validation accuracy: 67.8%
Minibatch loss at step 1500: 79.616272
Minibatch accuracy: 75.0%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 64.022667
Minibatch accuracy: 73.4%
Validation accuracy: 72.8%
Minibatch loss at step 2500: 50.750687
Minibatch accuracy: 79.7%
Validation accuracy: 73.2%
Minibatch loss at step 3000: 40.999378
Minibatch accuracy: 79.7%
Validation accuracy: 80.4%
Minibatch loss at step 3500: 32.770592
Minibatch accuracy: 83.6%
Validation accuracy: 80.1%
Minibatch loss at step 4000: 26.870134
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 4500: 22.396690
Minibatch accuracy: 78.1%
Validation accuracy: 84.2%
Minibatch loss at step 5000: 18.347321
Minibatch accuracy: 81.2%
Vali

In [104]:
batch_size = 128
layer1node = 1024
layernum=2
rate=0.5
lossrate= 5e-4
decaystep=1000
decayrate=0.96
graph = tf.Graph()
with graph.as_default():
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(rate, global_step, decaystep,decayrate)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layermiddle_weights = [tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1)) for i in range(layernum)]
  layermiddle_biases = [tf.Variable(tf.zeros([layer1node])) for i in range(layernum)]
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    
   # Model.
  def model(data,train=False):
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    for i in range(layernum):
        hidden=tf.nn.sigmoid(tf.matmul(hidden, layermiddle_weights[i]) + layermiddle_biases[i])
    if train:return tf.nn.dropout(tf.matmul(hidden, layer3_weights) + layer3_biases)
    else return tf.matmul(hidden, layer3_weights) + layer3_biases
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases)  + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
  for i in range(layernum):
    regularizers += (tf.nn.l2_loss(layermiddle_weights[i]) + tf.nn.l2_loss(layermiddle_biases[i]))

  logits = model(tf_train_dataset,True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss += lossrate * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 30001
# num_steps = 15001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 192.213669
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 500: 234.823853
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 185.209961
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 1500: 112.864944
Minibatch accuracy: 17.2%
Validation accuracy: 19.5%
Minibatch loss at step 2000: 69.685661
Minibatch accuracy: 64.8%
Validation accuracy: 63.6%
Minibatch loss at step 2500: 53.161961
Minibatch accuracy: 72.7%
Validation accuracy: 74.6%
Minibatch loss at step 3000: 42.440804
Minibatch accuracy: 76.6%
Validation accuracy: 81.8%
Minibatch loss at step 3500: 33.850895
Minibatch accuracy: 82.0%
Validation accuracy: 74.2%
Minibatch loss at step 4000: 27.645887
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%
Minibatch loss at step 4500: 22.413612
Minibatch accuracy: 80.5%
Validation accuracy: 84.6%
Minibatch loss at step 5000: 18.542557
Minibatch accuracy: 83.6%
Vali

In [ ]:
batch_size = 128
layer1node = 1024
layernum=2
rate=0.5
lossrate= 4e-4
decaystep=1000
decayrate=0.96
dropout = 0.75
graph = tf.Graph()
with graph.as_default():
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(rate, global_step, decaystep,decayrate)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layermiddle_weights = [tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1)) for i in range(layernum)]
  layermiddle_biases = [tf.Variable(tf.zeros([layer1node])) for i in range(layernum)]
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')

    
   # Model.
  def model(data,train=False):
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    if train:
        hidden = tf.nn.dropout(hidden,dropout)
    for i in range(layernum):
        hidden=tf.nn.sigmoid(tf.matmul(hidden, layermiddle_weights[i]) + layermiddle_biases[i])
        if train:
            hidden = tf.nn.dropout(hidden,dropout)
        
    result = tf.matmul(hidden, layer3_weights) + layer3_biases
    return result
  # Training computation.
  regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases)  + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
  for i in range(layernum):
    regularizers += (tf.nn.l2_loss(layermiddle_weights[i]) + tf.nn.l2_loss(layermiddle_biases[i]))

  logits = model(tf_train_dataset,True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
#   loss += lossrate * regularizers
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
num_steps = 35001
# num_steps = 15001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 27.763594
Minibatch accuracy: 5.5%
Validation accuracy: 10.0%